In [1]:
import sys, os

sys.path.insert(0, "../src")
from scraper.linkedin import Head_Hunter_9000

2023-09-17 18:45:32,520 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-17 18:45:32,521 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("jobboard")
2023-09-17 18:45:32,521 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-17 18:45:32,522 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("jobboard")
2023-09-17 18:45:32,523 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-17 18:45:32,524 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("jobquestion")
2023-09-17 18:45:32,524 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-17 18:45:32,525 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("jobquestion")
2023-09-17 18:45:32,526 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-17 18:45:32,527 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("optionsetoption")
2023-09-17 18:45:32,528 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-17 18:45:32,531 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("optionsetoption")
2023-09-17 18:45:32,532 INFO sqlalchemy.

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ChromeOptions
import html2text
from selenium.common.exceptions import NoSuchElementException
import urllib.parse
import configparser
import random
import time

import re
import data.architecture as da



# Logging in and getting some questions

In [3]:
hh_9000 = Head_Hunter_9000('/home/luca/Documents/Projects/Head_Hunter_9000/config.ini')
hh_9000.login()

https://www.linkedin.com/jobs/search/?f_AL=true&f_E=1%2C2%2C3&f_JT=F%2CC%2CT&f_WT=1%2C2%2C3&keywords=Data%20Analyst&location=United%20States


In [5]:
ext_job_id = hh_9000.get_job_id(hh_9000.driver.current_url)
job_info_container = hh_9000.driver.find_element(By.XPATH, "//div[@class='job-view-layout jobs-details']")
job_info = hh_9000.build_job_info(job_info_container, ext_job_id)

In [6]:
job_info

{'companyname': 'Visionaire Partners',
 'location': 'Birmingham, AL',
 'workplacetype': 'Hybrid',
 'postedtimeago': '2 weeks ago',
 'numapplicants': '359 applicants',
 'salary': '$75,000/yr - $95,000/yr · Full-time · Associate',
 'numemployees': '11-50 employees · Information Technology & Services',
 'jobtitle': 'Financial Data Analyst',
 'description': "**Financial Data Analyst (direct hire; only work in-office 2 days / month)**\n\nExciting opportunity to join a top-notch company as a Financial Data Analyst!\nYou will join our Finance team and leverage your analytical skills to derive\ninsights and improve our financial efficiency. You will be instrumental in\nhelping to build data-driven decision-making capabilities throughout the\norganization.\n\n  \n\nThis is a **direct hire** opportunity. You **must be able to attend onsite\nmeetings in Birmingham, AL approx. 2 days per month.** You can work remotely\nthe remainder of the time.\n\n  \n\nWe offer comprehensive benefits, bonus plan

In [7]:
easy_apply_button = job_info_container.find_element(By.XPATH, "//button[contains(@class, 'jobs-apply-button')]")
easy_apply_button.click()

In [8]:
from selenium.webdriver.support.ui import Select

def get_next_btn():
    nxt_btn = hh_9000.driver.find_elements(By.XPATH, "//span[text()='Next']/ancestor::button")
    if nxt_btn:
        return nxt_btn[0]
    
    rvw_btn = hh_9000.driver.find_elements(By.XPATH, "//span[text()='Review']/ancestor::button")
    if rvw_btn:
        return rvw_btn[0]
    
    return None


def fill_out_questions(freeresponse_question_containers, dropdown_question_containers, radiobutton_question_containers):
    for fr_q in freeresponse_question_containers:
        input_tag = fr_q.find_element(By.TAG_NAME, "input")
        input_tag.clear()
        input_tag.send_keys("1")

    for dd_q in dropdown_question_containers:
        select_el = dd_q.find_element(By.TAG_NAME, "select")
        select = Select(select_el)
        select.select_by_index(1)

    for rb_q in radiobutton_question_containers:
        radio_buttons = rb_q.find_elements(By.XPATH, ".//label")
        if radio_buttons:
            radio_buttons[0].click()


fr_prompts = []
dd_prompts_and_options = []
rb_prompts_and_options = []

next_btn = True
while next_btn:
    next_btn = get_next_btn()
    question_form = hh_9000.driver.find_element(By.XPATH, "//div[@class='pb4']")

    freeresponse_question_containers = question_form.find_elements(By.XPATH, "./div[contains(@class, 'jobs-easy-apply-form-section__grouping')]//div[@data-test-single-line-text-form-component]")
    dropdown_question_containers = question_form.find_elements(By.XPATH, "./div[contains(@class, 'jobs-easy-apply-form-section__grouping')]//div[@data-test-text-entity-list-form-component]")
    radiobutton_question_containers = question_form.find_elements(By.XPATH, "./div[contains(@class, 'jobs-easy-apply-form-section__grouping')]//fieldset[@data-test-form-builder-radio-button-form-component]")

    fill_out_questions(freeresponse_question_containers, dropdown_question_containers, radiobutton_question_containers)

    fr_prompts.extend(hh_9000.scrape_freeresponse_questions(freeresponse_question_containers))
    dd_prompts_and_options.extend(hh_9000.scrape_dropdown_questions(dropdown_question_containers))
    rb_prompts_and_options.extend(hh_9000.scrape_radiobutton_questions(radiobutton_question_containers))

    next_btn.click()


Have you completed the following level of education: Bachelor's Degree?
Have you completed the following level of education: Bachelor's Degree?
Will you now, or in the future, require sponsorship for employment visa status (e.g. H-1B visa status)?


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='pb4']"}
  (Session info: chrome=113.0.5672.126)
Stacktrace:
#0 0x5563b12f3133 <unknown>
#1 0x5563b1027966 <unknown>
#2 0x5563b10640dc <unknown>
#3 0x5563b1064211 <unknown>
#4 0x5563b109e6c4 <unknown>
#5 0x5563b1083bfd <unknown>
#6 0x5563b109c1c7 <unknown>
#7 0x5563b10839a3 <unknown>
#8 0x5563b105846a <unknown>
#9 0x5563b105955e <unknown>
#10 0x5563b12b2cae <unknown>
#11 0x5563b12b68fe <unknown>
#12 0x5563b12bff20 <unknown>
#13 0x5563b12b7923 <unknown>
#14 0x5563b128ac0e <unknown>
#15 0x5563b12dab08 <unknown>
#16 0x5563b12dac97 <unknown>
#17 0x5563b12eb113 <unknown>
#18 0x7f80dd659ea7 start_thread


In [9]:
dd_prompts_and_options

[('Email address',
  [{'text': 'Select an option', 'value': 'Select an option'},
   {'text': 'luca.guarro@gmail.com', 'value': 'luca.guarro@gmail.com'}]),
 ('Phone country code',
  [{'text': 'Select an option', 'value': 'Select an option'},
   {'text': 'United States (+1)', 'value': 'United States (+1)'},
   {'text': 'Afghanistan (+93)', 'value': 'Afghanistan (+93)'},
   {'text': 'Aland Islands (+358)', 'value': 'Aland Islands (+358)'},
   {'text': 'Albania (+355)', 'value': 'Albania (+355)'},
   {'text': 'Algeria (+213)', 'value': 'Algeria (+213)'},
   {'text': 'American Samoa (+1)', 'value': 'American Samoa (+1)'},
   {'text': 'Andorra (+376)', 'value': 'Andorra (+376)'},
   {'text': 'Angola (+244)', 'value': 'Angola (+244)'},
   {'text': 'Anguilla (+1)', 'value': 'Anguilla (+1)'},
   {'text': 'Antarctica (+0)', 'value': 'Antarctica (+0)'},
   {'text': 'Antigua and Barbuda (+1)', 'value': 'Antigua and Barbuda (+1)'},
   {'text': 'Argentina (+54)', 'value': 'Argentina (+54)'},
   {'te

# Insert Test

In [3]:
question_text = "Have you completed the following level of education: Bachelor's Degree?"
options = [{'text': 'Yes', 'value': 'Yes'},{'text': 'No', 'value': 'No'}]

In [4]:
import data.manager as dm

In [5]:
options_sa, was_option_created = dm.insert_options(options)

2023-09-17 18:45:44,111 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-17 18:45:44,116 INFO sqlalchemy.engine.Engine SELECT option.id AS option_id, option.text AS option_text, option.value AS option_value 
FROM option 
WHERE option.text = ? AND option.value = ?
 LIMIT ? OFFSET ?
2023-09-17 18:45:44,117 INFO sqlalchemy.engine.Engine [generated in 0.00130s] ('Yes', 'Yes', 1, 0)
no option found
2023-09-17 18:45:44,124 INFO sqlalchemy.engine.Engine INSERT INTO option (text, value) VALUES (?, ?)
2023-09-17 18:45:44,128 INFO sqlalchemy.engine.Engine [generated in 0.00364s] ('Yes', 'Yes')
2023-09-17 18:45:44,130 INFO sqlalchemy.engine.Engine SELECT option.id AS option_id, option.text AS option_text, option.value AS option_value 
FROM option 
WHERE option.text = ? AND option.value = ?
 LIMIT ? OFFSET ?
2023-09-17 18:45:44,131 INFO sqlalchemy.engine.Engine [cached since 0.01552s ago] ('No', 'No', 1, 0)
no option found


In [7]:
was_optionset_created = False
if was_option_created:
    optionset_sa = dm.make_optionset(options_sa)
    was_optionset_created = True
else:
    optionset_sa = dm.does_optionset_exist(options_sa)
    if not optionset_sa:
        optionset_sa = dm.make_optionset(options_sa)
        was_optionset_created = True

In [10]:
if was_optionset_created:
    question_sa = dm.create_question(question_text, da.QuestionType.RADIOBUTTON, optionset_sa)
else:
    question_sa = dm.does_question_exist(question_text, da.QuestionType.RADIOBUTTON, optionset_sa)
    if not question_sa:
        question_sa = dm.create_question(question_text, da.QuestionType.RADIOBUTTON, optionset_sa)

In [10]:
job_info = {'companyname': 'Visionaire Partners',
 'location': 'Birmingham, AL',
 'workplacetype': 'Hybrid',
 'postedtimeago': '2 weeks ago',
 'numapplicants': '359 applicants',
 'salary': '$75,000/yr - $95,000/yr · Full-time · Associate',
 'numemployees': '11-50 employees · Information Technology & Services',
 'jobtitle': 'Financial Data Analyst',
 'description': "**Financial Data Analyst (direct hire; only work in-office 2 days / month)**\n\nExciting opportunity to join a top-notch company as a Financial Data Analyst!\nYou will join our Finance team and leverage your analytical skills to derive\ninsights and improve our financial efficiency. You will be instrumental in\nhelping to build data-driven decision-making capabilities throughout the\norganization.\n\n  \n\nThis is a **direct hire** opportunity. You **must be able to attend onsite\nmeetings in Birmingham, AL approx. 2 days per month.** You can work remotely\nthe remainder of the time.\n\n  \n\nWe offer comprehensive benefits, bonus plan, 401k match, and continuing\neducation allowance.\n\n  \n\n **RESPONSIBILITIES:**\n\n  * Act as a consultant to our Finance Team to derive insights and improve our efficiency\n  * Analyze financial data related to identify patterns, trends, and opportunities for improvement\n  * Generate reports and visualizations to communicate data-driven insights to non-technical stakeholders in a clear and actionable manner\n  * Work closely with the Finance team to define and monitor key performance indicators (KPIs), ensuring financial objectives are being met\n  * Utilize data modeling and statistical analysis to predict future financial outcomes and drive strategic decision-making\n  * Partner with cross-functional teams to ensure data accuracy and consistent reporting\n  * Ensure compliance with data handling, storage, and privacy regulations\n  * Troubleshoot data-related issues and establish quality control procedures\n\n  \n\n**REQUIRED SKILLS:**\n\n  * Must be local to Birmingham, AL (attend onsite meetings 2 days/month)\n  * Bachelor's Degree\n  * 2 to 4 years as a Data Analyst (\n  * Experience with RDBMS Relational Databases (e.g., SQL Server, Oracle)\n  * Strong experience with Reporting Software (e.g., SSRS, Business Objects)\n  * Experience with Data Visualization software (e.g., Tableau, Power BI)\n  * Data Mining experience\n  * Excellent verbal and written communication skills\n\n  \n\n**PREFERRED SKILLS:**\n\n  * Insurance Claims industry experience\n\n  \n\n_W2 ONLY; No 3rd Parties/C2C/Visa Sponsorship_\n\n",
 'jobboardid': 'linkedin',
 'appsubmitted': False,
 'extjobid': '3701783986'}

In [20]:
question = "Have you completed the following level of education: Bachelor's Degree?"

In [16]:
try:
    # Query for a RadioButtonsQuestion with the given question text and optionsetid
    existing_question = session.query(da.RadioButtonQuestion).filter(
        da.RadioButtonQuestion.question == question,
        da.RadioButtonQuestion.optionset == new_optionset
    ).one()

    # If found, 'existing_question' will contain the question
    print(f"Question found: {existing_question}")
except NoResultFound:
    print("Yolo")
    # If no matching question is found, create a new RadioButtonsQuestion
    new_question = da.RadioButtonQuestion(
        question = question,
        optionset = new_optionset
    )

    # Add the new question to the session and commit to insert it
    session.add(new_question)
    session.commit()

2023-09-17 11:08:37,438 INFO sqlalchemy.engine.Engine SELECT radiobuttonquestion.id AS radiobuttonquestion_id, question.id AS question_id, question.question AS question_question, question.type AS question_type, radiobuttonquestion.optionsetid AS radiobuttonquestion_optionsetid, radiobuttonquestion.answerasoptionid AS radiobuttonquestion_answerasoptionid 
FROM question JOIN radiobuttonquestion ON question.id = radiobuttonquestion.id 
WHERE question.question = ? AND ? = radiobuttonquestion.optionsetid
2023-09-17 11:08:37,440 INFO sqlalchemy.engine.Engine [cached since 156.5s ago] ("Have you completed the following level of education: Bachelor's Degree?", 1)
Yolo
2023-09-17 11:08:37,447 INFO sqlalchemy.engine.Engine INSERT INTO question (question, type) VALUES (?, ?)
2023-09-17 11:08:37,448 INFO sqlalchemy.engine.Engine [generated in 0.00130s] ("Have you completed the following level of education: Bachelor's Degree?", 'radio buttons')
2023-09-17 11:08:37,450 INFO sqlalchemy.engine.Engine 

In [21]:
existing_question = session.query(da.RadioButtonQuestion).filter(
    da.RadioButtonQuestion.question == question,
    da.RadioButtonQuestion.optionset == matching_optionset
)

In [23]:
test = existing_question.first()

2023-09-17 16:59:56,227 INFO sqlalchemy.engine.Engine SELECT radiobuttonquestion.id AS radiobuttonquestion_id, question.id AS question_id, question.question AS question_question, question.type AS question_type, radiobuttonquestion.optionsetid AS radiobuttonquestion_optionsetid, radiobuttonquestion.answerasoptionid AS radiobuttonquestion_answerasoptionid 
FROM question JOIN radiobuttonquestion ON question.id = radiobuttonquestion.id 
WHERE question.question = ? AND ? = radiobuttonquestion.optionsetid
 LIMIT ? OFFSET ?
2023-09-17 16:59:56,229 INFO sqlalchemy.engine.Engine [cached since 183s ago] ("Havee you completed the following level of education: Bachelor's Degree?", 1, 1, 0)


In [27]:
if test:
    print(1)
else:
    print(2)

2


In [15]:
new_question = da.RadioButtonQuestion()


In [ ]:
    da.RadioButtonQuestion.question == question,
    da.RadioButtonQuestion.optionset == new_optionset

In [16]:
new_question

<Question(question='None', type='radio buttons')>